In [36]:
#importing libraries
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import VimeoVideo
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import Ridge
from sklearn.utils.validation import check_is_fitted

In [7]:
#taking in the data
df = pd.read_csv("restaurant_data.csv")
print(df.head())
print(df.info())

   order_id        date        item_name  item_type  item_price  quantity  \
0         1  07-03-2022         Aalopuri   Fastfood          20        13   
1         2   8/23/2022          Vadapav   Fastfood          20        15   
2         3  11/20/2022          Vadapav   Fastfood          20         1   
3         4  02-03-2023  Sugarcane juice  Beverages          25         6   
4         5  10-02-2022  Sugarcane juice  Beverages          25         8   

   transaction_amount transaction_type received_by time_of_sale  
0                 260              NaN         Mr.        Night  
1                 300             Cash         Mr.    Afternoon  
2                  20             Cash         Mr.    Afternoon  
3                 150           Online         Mr.        Night  
4                 200           Online         Mr.      Evening  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column              Non-Null C

In [8]:
#dropping columns that do not give any info
df.drop(columns=["order_id","received_by"], inplace=True)

In [9]:
#dropping values which don't have a transaction type
df = df.dropna(subset=["transaction_type"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 893 entries, 1 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date                893 non-null    object
 1   item_name           893 non-null    object
 2   item_type           893 non-null    object
 3   item_price          893 non-null    int64 
 4   quantity            893 non-null    int64 
 5   transaction_amount  893 non-null    int64 
 6   transaction_type    893 non-null    object
 7   time_of_sale        893 non-null    object
dtypes: int64(3), object(5)
memory usage: 62.8+ KB


In [45]:
#fixing the date format and creating a day of the week column
df["date"] = pd.to_datetime(df["date"],format='mixed',dayfirst=False)
df['day_of_week'] = df['date'].dt.day_name()
display(df.head())
df.to_csv("clean.csv", index=False)

,date,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,time_of_sale,day_of_week
1,2022-08-23,Vadapav,Fastfood,20,15,300,Cash,Afternoon,Tuesday
2,2022-11-20,Vadapav,Fastfood,20,1,20,Cash,Afternoon,Sunday
3,2023-02-03,Sugarcane juice,Beverages,25,6,150,Online,Night,Friday
4,2022-10-02,Sugarcane juice,Beverages,25,8,200,Online,Evening,Sunday
5,2022-11-14,Vadapav,Fastfood,20,10,200,Cash,Evening,Monday


In [11]:
df.groupby("item_name")[["transaction_amount", "quantity"]].sum().sort_values(by="transaction_amount", ascending=False)
#high demand for coffee and cane juice, incr price to inc rev, introduce more bevs

,transaction_amount,quantity
item_name,,
Sandwich,58080,968
Frankie,52650,1053
Cold coffee,45680,1142
Sugarcane juice,28025,1121
Panipuri,22380,1119
Aalopuri,18560,928
Vadapav,18060,903


In [12]:
df.groupby("day_of_week")["transaction_amount"].mean().sort_values()
#maybe launch a wednesday offer to boost wednesday revenue

,transaction_amount
day_of_week,
Wednesday,251.048951
Saturday,253.082707
Sunday,270.073529
Friday,270.853659
Monday,283.266129
Thursday,291.487603
Tuesday,295.884956


In [42]:
daily_revenue = df.groupby("date")["transaction_amount"].sum().reset_index()
daily_revenue['day_of_week'] = daily_revenue['date'].dt.day_name()
daily_revenue

,date,transaction_amount,day_of_week
0,2022-04-01,420,Friday
1,2022-04-02,1500,Saturday
2,2022-04-03,480,Sunday
3,2022-04-04,1900,Monday
4,2022-04-05,100,Tuesday
...,...,...,...
334,2023-03-25,840,Saturday
335,2023-03-26,280,Sunday
336,2023-03-27,1040,Monday
337,2023-03-29,1080,Wednesday


In [46]:
daily_revenue["date"] = pd.to_datetime(daily_revenue["date"]) # Create a "month" column
daily_revenue["month"] = daily_revenue["date"].dt.to_period("M") # Aggregate by month
monthly_revenue = daily_revenue.groupby("month")["transaction_amount"].sum().reset_index() # Convert period back to timestamp for plotting
monthly_revenue["month"] = monthly_revenue["month"].dt.to_timestamp()
monthly_revenue.to_csv("monthly.csv", index=False)

In [47]:
sales_by_time_of_day = df.groupby("time_of_sale")["transaction_amount"].sum().reset_index()
display(sales_by_time_of_day)
#not a lot of variation

,time_of_sale,transaction_amount
0,Afternoon,49155
1,Evening,47505
2,Midnight,46425
3,Morning,46910
4,Night,53440


In [48]:
daily_revenue["transaction_amount"].mean()

np.float64(718.0973451327434)